In [ ]:
# Libraries
import numpy as np
import pandas as pd
import requests as re
from requests.auth import HTTPBasicAuth
import nltk
import sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
import os
# to get reddit comments, use praw
!pip install praw
import praw
!pip install asyncpraw
import asyncpraw
import asyncio

In [ ]:
from asyncpraw.models import MoreComments

In [ ]:
# Reddit logins:
# VOk3XO5hfhdhUGATseCgtg - App ID
# ISVVO9hObXskwjhmQODtw9NdMLWv6w - secret

In [ ]:
# Adding my reddit credentials to pull from
app_id = "VOk3XO5hfhdhUGATseCgtg"
secret = "ISVVO9hObXskwjhmQODtw9NdMLWv6w"
username = 'TheMuskyHairbrush'
password = 'SecretStoryKeeper888!'

In [ ]:
# Access reddit using praw to create an authorized instance
reddit = asyncpraw.Reddit(
    client_id = app_id,
    client_secret = secret,
    password = password,
    user_agent ="UniversalHealthcareSentimentScript",
    username = username,
)

print(reddit.read_only)

False


In [ ]:
# The reddit.read_only checks to make sure I can access subreddits and do detailed stuff. That's what the authint is for.
# If it were true, I wouldn't be able to do detailed stuff.

In [ ]:
# Next, I want to showcase the raw data of reddit comments using this AskReddit post about what would happen if the U.S. got Universal Healthcare the Next Day: https://www.reddit.com/r/AskReddit/comments/1io4svg/what_will_happen_if_us_switched_to_universal/
# Warning: Profanity

In [ ]:
# Get comments from a post
# Reddit PRAW API returns a list of comment objects as raw data
# Posts have a unique string id before the title in the url, use that to id
post1 = '1io4svg'
# Get the specific submission
submission = await reddit.submission("1io4svg")
# Get the comments, the replace_more allows me to gather comments under the replies of a comment
await submission.comments.replace_more(limit=None)
# This for loop will print all of the comments in this askreddit post, doing this as a way to showcase raw data
for comment in submission.comments.list():
    print(comment.body)
    print(type(comment))

The ones opposing it today will be the first to get themselves checked.
<class 'asyncpraw.models.reddit.comment.Comment'>
Possible all the rich insurance company's go out of business.   Pharmaceuticals companies also wouldn't be able to charged 2 arms 2 legs and a first born for drugs.
<class 'asyncpraw.models.reddit.comment.Comment'>
Probably need to check with Dr. Strange to see what alternate universe we're actually in.
<class 'asyncpraw.models.reddit.comment.Comment'>
So where I live, finding a new PCP and setting up an appointment takes ~6 months. When more people can access healthcare, that time will likely increase. Of course anyone seeking medical assistance should be able to access it, but the US isn't ready to handle it just yet. Plus health insurance companies and drug manufacturers are making billions off of us. Any sudden change would mess up the entire system.

I strongly believe that the US should have universal healthcare, but it won't be easy to switch to it overnight.

In [ ]:
# Now to actually pull and label Reddit comment data
# Not focusing on date/time, and the url and commenter aren't needed, they're too unique
# Only pulling comment, and score. Score could be useful for sentiment
# Pulling from this AskConservatives post on opinions about Universal Healthcare: https://www.reddit.com/r/AskConservatives/comments/1fcsij5/for_those_who_are_why_are_you_personally_opposed/
submission = await reddit.submission("1fcsij5")
# Get the comments, the replace_more allows me to gather comments under the replies of a comment
await submission.comments.replace_more(limit=None)

# Initialize a list to store the comments to turn into a dataframe
comment_list1 = []
# This for loop will add the comment and score to the list as a dictionary, plus a label
for comment in submission.comments.list():
    comment_list1.append({
            'Comment': comment.body,
            'Score': comment.score,
            'LABEL': 'Conservative'
        })

In [ ]:
# Check if it worked
print(comment_list1[0])

{'Comment': 'Please use [Good Faith](https://www.reddit.com/r/AskConservatives/comments/107i33m/announcement_rule_7_good_faith_is_now_in_effect) and the [Principle of Charity](https://en.wikipedia.org/wiki/Principle_of_charity) when commenting. Gender issues are only allowed on Wednesdays. Antisemitism and calls for violence will not be tolerated, especially when [discussing the Israeli-Palestinian conflict](https://www.reddit.com/r/AskConservatives/comments/17ygktl/antisemitism_askconservative_and_you/).\n\n\n*I am a bot, and this action was performed automatically. Please [contact the moderators of this subreddit](/message/compose/?to=/r/AskConservatives) if you have any questions or concerns.*', 'Score': 1, 'LABEL': 'Conservative'}


In [ ]:
#I'm actually going to drop that, it's an automod comment and not needed
comment_list1 = comment_list1[1:]

In [ ]:
# Check again
print(comment_list1[0])

{'Comment': "I personally am not, but what I ussually hear and I understand it completely is everything the Government gets involved in turns to shit, has miles of red tape, is wasteful and inefficient.  There are also complexities with moral issues such as the government funding abortions, genital mutiliation, etc and people don't want their tax money going to that.\n\nNow I personally believe those those are mostly are true of insurance as well and I'd rather have the US Government which doesn't have an incentive for income (like a private company) would be a more cost effective option for the tax payer...also the fact that literally every other western nation has it and on top of that the US spends more on healthcare per capita then almost all of those nations and we get less.  I really do think there is a bit of a turning point on the healthcare conversation as even young conservatives who are aware of the world are starting to see healthcare differently especially once they have f

In [ ]:
# Great. Now to turn those comments into a dataframe
Reddit_DF = pd.DataFrame(comment_list1)

In [ ]:
Reddit_DF.head()

,Comment,Score,LABEL
0,"I personally am not, but what I ussually hear ...",63,Conservative
1,"The real question is, how do I know those taxe...",11,Conservative
2,I’m not opposed to a universal system (we’re v...,4,Conservative
3,Why put a bandaid on an artery bleed without a...,15,Conservative
4,I don't trust the federal government running t...,10,Conservative


In [ ]:
# Repeat with other posts
# Comes from an r/AskALiberal Post: https://www.reddit.com/r/AskALiberal/comments/1ggyzl5/how_far_would_you_go_with_universal_healthcare/
submission = await reddit.submission("1ggyzl5")
# Get the comments, the replace_more allows me to gather comments under the replies of a comment
# However, setting the limit to 0 gets only the top-level comments, limiting the replies and making sure i get the most relevant ones
# It's a way to stretch my ratelimit
await submission.comments.replace_more(limit=0)

# Initialize a list to store the comments to turn into a dataframe
comment_list2 = []
# This for loop will add the comment and score to the list as a dictionary, plus a label
for comment in submission.comments.list():
    comment_list2.append({
            'Comment': comment.body,
            'Score': comment.score,
            'LABEL': 'Liberal'
        })

In [ ]:
print(comment_list2[0])

{'Comment': 'The following is a copy of the original post to record the post as it was originally written.\n\nFor those in favor of universal Healthcare, curious where you would draw the line regarding what is covered, and why.\n\nInteresting examples to consider include IVF/fertility treatments, dental care, optical, lasik, over the counter meds like Tylenol, cough/cold meds, first aid supplies, glp1s for obesity, therapy/counseling, home health nursing care, long term acute care. Do you see a role for cost sharing / subsidization for any of this? \n\nMy impression is that a lot of this cohort feels strongly that healthcare is a human right, and I think it could be an interesting discussion to chat about the scope of healthcare as it pertains to this. \n\n*I am a bot, and this action was performed automatically. Please [contact the moderators of this subreddit](/message/compose/?to=/r/AskALiberal) if you have any questions or concerns.*', 'Score': 1, 'LABEL': 'Liberal'}


In [ ]:
Reddit_DF2 = pd.DataFrame(comment_list2)

In [ ]:
Reddit_DF2.head()

,Comment,Score,LABEL
0,The following is a copy of the original post t...,1,Liberal
1,I think American progressives are overly obses...,15,Liberal
2,Let's first clarify that universal healthcare ...,7,Liberal
3,The legal basis I am familiar with specify tha...,3,Liberal
4,I would expect it to cover everything you’ve l...,3,Liberal


In [ ]:
Reddit_DF2 = Reddit_DF2.iloc[1:]

In [ ]:
Reddit_DF2.head()

,Comment,Score,LABEL
1,I think American progressives are overly obses...,15,Liberal
2,Let's first clarify that universal healthcare ...,7,Liberal
3,The legal basis I am familiar with specify tha...,3,Liberal
4,I would expect it to cover everything you’ve l...,3,Liberal
5,Everyone is completely covered for any issues ...,3,Liberal


In [ ]:
# Pull more comments
# AskReddit: Should The U.S. Have Universal Healthcare? https://www.reddit.com/r/AskReddit/comments/liguwt/should_the_us_have_universal_healthcare_like/
submission = await reddit.submission("liguwt")
await submission.comments.replace_more(limit = None)

# Initialize a list to store the comments to turn into a dataframe
comment_list3 = []
# This for loop will add the comment and score to the list as a dictionary, plus a label
for comment in submission.comments.list():
    comment_list3.append({
            'Comment': comment.body,
            'Score': comment.score,
            'LABEL': 'AskReddit'
        })

In [ ]:
print(comment_list3[0])

{'Comment': "I'm Canadian and let me tell you that I have never once had to put off getting care because I couldn't afford it. \n\n I come from a working class family. If we had to pay for out healthcare, my parents would never see doctors or go to the hospital, even with my dad's congenital heart issues, it would be too expensive for them. Socialized medicine saves lives and most of us don't mind paying a few extra dollars in taxes for that safety net.", 'Score': 39, 'LABEL': 'AskReddit'}


In [ ]:
Reddit_DF3 = pd.DataFrame(comment_list3)
Reddit_DF3.head()

,Comment,Score,LABEL
0,I'm Canadian and let me tell you that I have n...,39,AskReddit
1,yeah. \n\npeople claim that we wait hours in t...,8,AskReddit
2,Yes. It will result in better healthcare that...,6,AskReddit
3,If every other major first world country can d...,17,AskReddit
4,The US should have universal healthcare for on...,15,AskReddit


In [ ]:
# Repeat for more comments
# AskAConservative Post, Have you changed your mind on Universal Healthcare: https://www.reddit.com/r/AskConservatives/comments/1fcr7z9/have_conservatives_changed_their_opinions_on/
submission = await reddit.submission("1fcr7z9")

await submission.comments.replace_more(limit = None)

# Initialize a list to store the comments to turn into a dataframe
comment_list4 = []
# This for loop will add the comment and score to the list as a dictionary, plus a label
for comment in submission.comments.list():
    comment_list4.append({
            'Comment': comment.body,
            'Score': comment.score,
            'LABEL': 'Conservative'
        })


In [ ]:
print(comment_list4[0])

{'Comment': 'Please use [Good Faith](https://www.reddit.com/r/AskConservatives/comments/107i33m/announcement_rule_7_good_faith_is_now_in_effect) and the [Principle of Charity](https://en.wikipedia.org/wiki/Principle_of_charity) when commenting. Gender issues are only allowed on Wednesdays. Antisemitism and calls for violence will not be tolerated, especially when [discussing the Israeli-Palestinian conflict](https://www.reddit.com/r/AskConservatives/comments/17ygktl/antisemitism_askconservative_and_you/).\n\n\n*I am a bot, and this action was performed automatically. Please [contact the moderators of this subreddit](/message/compose/?to=/r/AskConservatives) if you have any questions or concerns.*', 'Score': 1, 'LABEL': 'Conservative'}


In [ ]:
# I always check the first entry because archived posts frequently have reminder messages, and many subreddits use an automod. It's hard to tell off the bat, so it's worth checking manually.
comment_list4 = comment_list4[1:]

In [ ]:
Reddit_DF4 = pd.DataFrame(comment_list4)
Reddit_DF4.head()

,Comment,Score,LABEL
0,"As a conservative, this probably puts me in th...",10,Conservative
1,It is not as simple as Public Option vs Privat...,6,Conservative
2,After the last try failed so badly are we will...,2,Conservative
3,[removed],1,Conservative
4,The problem in this whole argument is no one w...,1,Conservative


In [ ]:
# I want at least one more liberal subreddit response, and one more askreddit for now
# AskALiberal Post: Why wouldn't universal healthcare work in the United States: https://www.reddit.com/r/AskALiberal/comments/rsjyey/why_couldnt_universal_healthcare_work_in_the/

submission = await reddit.submission("rsjyey")

await submission.comments.replace_more(limit = None)

# Initialize a list to store the comments to turn into a dataframe
comment_list5 = []
# This for loop will add the comment and score to the list as a dictionary, plus a label
for comment in submission.comments.list():
    comment_list5.append({
            'Comment': comment.body,
            'Score': comment.score,
            'LABEL': 'Liberal'
        })

In [ ]:
print(comment_list5[0])

{'Comment': 'The following is a copy of the original post to record the post as it was originally written.\n\nThis question is directed to liberals who don’t think a universal healthcare system or a single payer system could work in the United States. \n\nI find it odd that a nation that celebrates its exceptional character suddenly becomes very weak at looking after its own citizens. \n\n*I am a bot, and this action was performed automatically. Please [contact the moderators of this subreddit](/message/compose/?to=/r/AskALiberal) if you have any questions or concerns.*', 'Score': 1, 'LABEL': 'Liberal'}


In [ ]:
comment_list5 = comment_list5[1:]

In [ ]:
Reddit_DF5 = pd.DataFrame(comment_list5)
Reddit_DF5.head()

,Comment,Score,LABEL
0,"The only reason it wouldn't, is the same reaso...",123,Liberal
1,"It could. Honestly, we could pick almost any s...",44,Liberal
2,Because half the country is determined to make...,61,Liberal
3,Universal=/=single payer.\n\nSingle payer is t...,21,Liberal
4,As an Australian who has always lived with uni...,7,Liberal


In [ ]:
# AskReddit: Americans, do you want Universal Healthcare? https://www.reddit.com/r/AskReddit/comments/piok3m/americans_do_you_want_universal_healthcare_whywhy/

submission = await reddit.submission("piok3m")

# This is a HUGE post, over 16k comments. Gotta limit responses
await submission.comments.replace_more(limit = 0)

# Initialize a list to store the comments to turn into a dataframe
comment_list6 = []
# This for loop will add the comment and score to the list as a dictionary, plus a label
for comment in submission.comments.list():
    comment_list6.append({
            'Comment': comment.body,
            'Score': comment.score,
            'LABEL': 'AskReddit'
        })

In [ ]:
comment_list6[0]

{'Comment': "I had a staph infection on one of my toes.\n\nI ignored it.\n\nI ignored it until my foot was almost the size of a football(american) and half my foot was black.\n\nI couldn't afford to take time off, so I kept working when it started. I worked the night I finally asked a friend to take me to the emergency room. \n\nThe only reason I decided to go was because I got *extremely* drunk, like 3/4 of a 1.5 liter bottle of Sailor Jerry's drunk. ( I was worried about the $)\n\nAnyways. I got x-rayed, stuck in the magnet tube and had a bunch of tests ran.\n\nMy main Doc told me that I had to choose between half of my foot, or half of my leg getting amputated.\n\nI chose half foot.\n\nWhen I woke up, I still had my foot. \n\nSomething about the infection not getting into my bone.\n\nDoc also mentioned that there was evidence of maggots when he abraded the corrupted flesh off of me.\n\nI was in the hospital after that for a week. My bill was almost 200k USD.\n\n\n*yes guys, I was st

In [ ]:
Reddit_DF6 = pd.DataFrame(comment_list6)
Reddit_DF6.head()

,Comment,Score,LABEL
0,I had a staph infection on one of my toes.\n\n...,4649,AskReddit
1,"My current job is slowly driving me insane, bu...",8035,AskReddit
2,I had a pretty well-paid job making sure the h...,9102,AskReddit
3,Yes. Because cancer is a bitch. I had good ins...,6077,AskReddit
4,Yes. I want my relationship with my work to ju...,1778,AskReddit


In [ ]:
# Time to concat all of these into a base dataframe.
# Stack all the rows via pandas concat
Raw_Reddit_DF = pd.concat([Reddit_DF, Reddit_DF2, Reddit_DF3, Reddit_DF4, Reddit_DF5, Reddit_DF6], ignore_index = True)

Raw_Reddit_DF.head()

,Comment,Score,LABEL
0,"I personally am not, but what I ussually hear ...",63,Conservative
1,"The real question is, how do I know those taxe...",11,Conservative
2,I’m not opposed to a universal system (we’re v...,4,Conservative
3,Why put a bandaid on an artery bleed without a...,15,Conservative
4,I don't trust the federal government running t...,10,Conservative


In [ ]:
# Get a feel for how many rows of data we have
Raw_Reddit_DF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1470 entries, 0 to 1469
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Comment  1470 non-null   object
 1   Score    1470 non-null   int64 
 2   LABEL    1470 non-null   object
dtypes: int64(1), object(2)
memory usage: 34.6+ KB


In [ ]:
# Check how many comments from each subreddit
Raw_Reddit_DF['LABEL'].value_counts()

,count
LABEL,
AskReddit,662
Conservative,475
Liberal,333


In [ ]:
# I tried saving it as a CSV at first, but the punctuation translated badly. Gonna remove them
Raw_Reddit_DF['Comment'] = Raw_Reddit_DF['Comment'].str.replace(r'[^a-zA-Z0-9\s]', '', regex=True)

In [ ]:
Raw_Reddit_DF.head()

,Comment,Score,LABEL
0,I personally am not but what I ussually hear a...,63,Conservative
1,The real question is how do I know those taxes...,11,Conservative
2,Im not opposed to a universal system were very...,4,Conservative
3,Why put a bandaid on an artery bleed without a...,15,Conservative
4,I dont trust the federal government running th...,10,Conservative


In [ ]:
# Next, lowercase all the text
Raw_Reddit_DF['Comment'] = Raw_Reddit_DF['Comment'].str.lower()

In [ ]:
Raw_Reddit_DF.head()

,Comment,Score,LABEL
0,i personally am not but what i ussually hear a...,63,Conservative
1,the real question is how do i know those taxes...,11,Conservative
2,im not opposed to a universal system were very...,4,Conservative
3,why put a bandaid on an artery bleed without a...,15,Conservative
4,i dont trust the federal government running th...,10,Conservative


In [ ]:
# Now, to intialize a raw CSV file of data
Raw_Reddit_DF.to_csv('Raw Reddit Data.csv', index = False)